In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.optimize import linear_sum_assignment
import scipy.io as sio
import numpy as np
import pandas as pd
import os

In [2]:
def numpy_SAD(y_true, y_pred):
    return np.arccos(y_pred.dot(y_true) / (np.linalg.norm(y_true) * np.linalg.norm(y_pred)))

def order_endmembers(endmembers, endmembersGT):
    num_endmembers = endmembers.shape[0]
    dict = {}
    sad_mat = np.ones((num_endmembers, num_endmembers))
    for i in range(num_endmembers):
        endmembers[i, :] = endmembers[i, :] / endmembers[i, :].max()
        endmembersGT[i, :] = endmembersGT[i, :] / endmembersGT[i, :].max()
    for i in range(num_endmembers):
        for j in range(num_endmembers):
            sad_mat[i, j] = numpy_SAD(endmembers[i, :], endmembersGT[j, :])
    rows = 0
    while rows < num_endmembers:
        minimum = sad_mat.min()
        index_arr = np.where(sad_mat == minimum)
        if len(index_arr) < 2:
            break
        index = (index_arr[0][0], index_arr[1][0])
        if index[0] in dict.keys():
            sad_mat[index[0], index[1]] = 100
        elif index[1] in dict.values():
            sad_mat[index[0], index[1]] = 100
        else:
            dict[index[0]] = index[1]
            sad_mat[index[0], index[1]] = 100
            rows += 1
    ASAM = 0
    num = 0
    for i in range(num_endmembers):
        if np.var(endmembersGT[dict[i]]) > 0:
            ASAM = ASAM + numpy_SAD(endmembers[i, :], endmembersGT[dict[i]])
            num += 1

    return dict, ASAM / float(num)

# Function to calculate SAD between two endmember vectors
def calculate_sad(endmember1, endmember2):
    endmember1 = endmember1 / np.linalg.norm(endmember1)
    endmember2 = endmember2 / np.linalg.norm(endmember2)
    sad_value = np.arccos(np.clip(np.dot(endmember1, endmember2), -1.0, 1.0))
    return sad_value

# Function to find the optimal ordering of predicted endmembers using SAD
def match_abundances(predicted, true):
    num = true.shape[2]
    cost_matrix = np.zeros((num, num))
    for i in range(num):
        for j in range(num):
            cost_matrix[i, j] = mean_squared_error(predicted[:, :, i], true[:, :, j])
    # print(cost_matrix)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    reordered_predicted = predicted[col_ind]
    return reordered_predicted, row_ind, col_ind

In [3]:
# List of folders/experiments (you can add more folders as needed)
folder_names = [
    # 'Original Results',
    'Blur (sigma = [0.1, 1.0])',
    'Blur (sigma = [0.1, 2.0])',
    'Crop (50)',
    'Crop (75)',
    'Crop (95)',
    'Vertical Flip',
    'Horizontal Flip',
    'Mix Flip',
    'Random Flip',
    'Jitter', 
    'Crop + Blur',
    'Crop + Flip',
    'Crop + Flip + Blur',
    'Crop + Jitter',
    'Crop + Jitter + Blur',
    'Crop + Jitter + Flip',
    'Jitter + Flip',
    'Jitter + Flip + Blur',
]

exp_names = [
    # '',
    'Results (25 runs, no seed, diff images)',
    'Results (25 runs, no seed, same images)',
    'Results (25 runs, seed, same images)'
]

datasetnames = {
    'Samson': {
        'num_endmembers': 3,
    },
    'Urban4': {
        'num_endmembers': 4,
    },
    'Urban5': {
        'num_endmembers': 5,
    },
    'Urban6': {
        'num_endmembers': 6,
    },
    'Cuprite_fixed': {
        'num_endmembers': 12,
    },
    'JasperRidge': {
        'num_endmembers': 4,
    },
}

# Variables to store results for each folder
results = []

# Loop over each dataset
for dataset in datasetnames:
    
    print(f'Processing dataset: {dataset}')

    path = "./Datasets/" + dataset + ".mat"

    try:
        data = sio.loadmat(path)
    except NotImplementedError:
        data = hdf.File(path, 'r')

    Y = np.asarray(data['Y'], dtype=np.float32)
    GT = np.asarray(data['GT'], dtype=np.float32)
    if Y.shape[0] < Y.shape[1]:
        Y = Y.transpose()
    Y = Y / np.max(Y.flatten())
    n_bands = Y.shape[1]
    n_rows = data['lines'].item()
    n_cols = data['cols'].item()
    Y = np.reshape(Y, (n_cols, n_rows, n_bands))
    
    if 'S_GT' in data.keys():
        S_GT = np.asarray(data['S_GT'], dtype=np.float32)
    else:
        S_GT = None
    
    num_endmembers = datasetnames[dataset]['num_endmembers']

    n = num_endmembers // 2  # how many digits we will display

    # Loop over each experiment folder 
    for exp in exp_names:

        # Loop over each folder and process the results
        for folder in folder_names:
            
            # Path template for the current folder (assuming 25 runs per folder)
            mat_file_template = f'./{dataset}/90 images/{exp}/{folder}/results_run_{{}}.mat'  # Path template for .mat files
            # mat_file_template = f'./Results/CNNAEU/{dataset}/synthetic_run{{}}.mat_run{{}}.mat'  # Path template for .mat files
            
            sad_values_per_run = []
            mse_values_per_run = []

            num_runs = 25
                
            for i in range(num_runs):
                # Load the .mat file for each run
                mat_file_path = mat_file_template.format(i + 1)
                # mat_file_path = mat_file_template.format(i, i + 1)
                
                # Check if the file exists before attempting to load
                if not os.path.exists(mat_file_path):
                    # If the file doesn't exist, skip this iteration
                    continue
        
                data = sio.loadmat(mat_file_path)
                
                predicted_endmembers = data.get('endmembers').T  # Predicted endmembers from the autoencoder
                # predicted_endmembers = data.get('M')  # Predicted endmembers from the autoencoder
                true_endmembers = GT  # Ground truth endmembers
                num_endmembers = true_endmembers.shape[0]

                for m in range(num_endmembers):
                    predicted_endmembers[m, :] = predicted_endmembers[m, :] / predicted_endmembers[m, :].max()
                    true_endmembers[m, :] = true_endmembers[m, :] / true_endmembers[m, :].max()
                
                if predicted_endmembers is not None and true_endmembers is not None:
                    order_dict, _ = order_endmembers(true_endmembers, predicted_endmembers)

                    reordered_predicted_endmembers = predicted_endmembers[[order_dict[k] for k in sorted(order_dict.keys())]]
                    
                    run_sad_values = []
                    # Calculate SAD for each matched endmember pair
                    for j in range(true_endmembers.shape[0]):
                        sad_value = numpy_SAD(true_endmembers[j], reordered_predicted_endmembers[j])
                        run_sad_values.append(sad_value)
                    
                    sad_values_per_run.append(run_sad_values)
                
                predicted_abundances = data.get('abundances')  # Predicted abundances
                # predicted_abundances = data.get('A')  # Predicted abundances
                true_abundances = S_GT  # Ground truth abundances
                
                if predicted_abundances is not None and true_abundances is not None:
                    run_mse_values = []
                    reordered_predicted_abundances = predicted_abundances[:, :, [order_dict[i] for i in sorted(order_dict.keys())]]
                    for l in range(true_abundances.shape[2]):
                        mse_value = mean_squared_error(reordered_predicted_abundances[:, :, l], true_abundances[:, :, l].T)
                        run_mse_values.append(mse_value)
                    
                    mse_values_per_run.append(run_mse_values)

            # Compute average and standard deviation for SAD and MSE
            average_sad_per_endmember = np.mean(sad_values_per_run, axis=0) if sad_values_per_run else None
            average_mse_per_abundance = np.mean(mse_values_per_run, axis=0) if mse_values_per_run else None
            std_sad_per_endmember = np.std(sad_values_per_run, axis=0) if sad_values_per_run else None
            std_mse_per_abundance = np.std(mse_values_per_run, axis=0) if mse_values_per_run else None
            
            # Store the results for this folder
            results.append({
                'Dataset': dataset,
                'Experiment': exp,
                'Folder': folder,
                'Average_SAD': average_sad_per_endmember,
                'Average_MSE': average_mse_per_abundance,
                'STD_SAD': std_sad_per_endmember,
                'STD_MSE': std_mse_per_abundance
            })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
print(results_df)

# Optionally, you can save the DataFrame to a CSV file
# results_df.to_csv('original_results_last_datasets.csv', index=False)
results_df.to_csv('comparison_results_90_images.csv', index=False)

Processing dataset: Samson
Processing dataset: Urban4
Processing dataset: Urban5
Processing dataset: Urban6
Processing dataset: Cuprite_fixed
Processing dataset: JasperRidge
         Dataset                               Experiment  \
0         Samson  Results (25 runs, no seed, diff images)   
1         Samson  Results (25 runs, no seed, diff images)   
2         Samson  Results (25 runs, no seed, diff images)   
3         Samson  Results (25 runs, no seed, diff images)   
4         Samson  Results (25 runs, no seed, diff images)   
..           ...                                      ...   
319  JasperRidge     Results (25 runs, seed, same images)   
320  JasperRidge     Results (25 runs, seed, same images)   
321  JasperRidge     Results (25 runs, seed, same images)   
322  JasperRidge     Results (25 runs, seed, same images)   
323  JasperRidge     Results (25 runs, seed, same images)   

                        Folder  \
0    Blur (sigma = [0.1, 1.0])   
1    Blur (sigma = [0.1, 

In [ ]:
# Filter valid SAD results and align with folder names
sad_boxplot_data = []
sad_labels = []

for i, sad_data in enumerate(results_df['Average_SAD']):
    if sad_data is not None:
        try:
            # Ensure sad_data is iterable and valid
            sad_boxplot_data.append(sad_data)  # Append SAD data (full list, not mean here)
            sad_labels.append(results_df['Folder'].iloc[i])  # Append corresponding folder name
        except Exception as e:
            print(f"Error processing SAD data for index {i}: {e}")

# Check lengths of data and labels
print(f"Number of boxplot data points: {len(sad_boxplot_data)}")
print(f"Number of labels: {len(sad_labels)}")

# Ensure lengths match
if len(sad_boxplot_data) != len(sad_labels):
    print("Mismatch detected between data and labels! Fixing...")
    min_length = min(len(sad_boxplot_data), len(sad_labels))
    sad_boxplot_data = sad_boxplot_data[:min_length]
    sad_labels = sad_labels[:min_length]

# Plot SAD boxplot
plt.figure(figsize=(12, 6))
plt.boxplot(sad_boxplot_data, tick_labels=sad_labels, showmeans=True)  # Using tick_labels for compatibility
plt.xticks(rotation=45, ha='right')
plt.title('SAD Distribution Across Folders (25 runs, no seed, diff images per run)')
plt.ylabel('SAD')
plt.tight_layout()
plt.show()

In [ ]:
# Filter valid MSE results and align with folder names
mse_boxplot_data = []
mse_labels = []

for i, mse_data in enumerate(results_df['Average_MSE']):
    if mse_data is not None:
        try:
            # Ensure mse_data is iterable and valid
            mse_boxplot_data.append(mse_data)  # Append MSE data (full list, not mean here)
            mse_labels.append(results_df['Folder'].iloc[i])  # Append corresponding folder name
        except Exception as e:
            print(f"Error processing MSE data for index {i}: {e}")

# Check lengths of data and labels
print(f"Number of boxplot data points: {len(mse_boxplot_data)}")
print(f"Number of labels: {len(mse_labels)}")

# Ensure lengths match
if len(mse_boxplot_data) != len(mse_labels):
    print("Mismatch detected between data and labels! Fixing...")
    min_length = min(len(mse_boxplot_data), len(mse_labels))
    mse_boxplot_data = mse_boxplot_data[:min_length]
    mse_labels = mse_labels[:min_length]

# Plot MSE boxplot
plt.figure(figsize=(12, 6))
plt.boxplot(mse_boxplot_data, tick_labels=mse_labels, showmeans=True)  # Using tick_labels for compatibility
plt.xticks(rotation=45, ha='right')
plt.title('MSE Distribution Across Folders (25 runs, no seed, diff images per run)')
plt.ylabel('MSE')
plt.tight_layout()
plt

In [ ]:
def reorder_predicted_endmembers(predicted_endmembers, true_endmembers):
    """
    Reorder predicted endmembers to match the true endmembers based on the `order_endmembers` function.
    
    Parameters:
        predicted_endmembers (numpy array): Predicted endmember matrix (num_endmembers, num_bands).
        true_endmembers (numpy array): True endmember matrix (num_endmembers, num_bands).
    
    Returns:
        numpy array: Reordered predicted endmembers.
    """
    order_dict, _ = order_endmembers(true_endmembers, predicted_endmembers)
    reordered_endmembers = predicted_endmembers[[order_dict[k] for k in sorted(order_dict.keys())], :]
    return reordered_endmembers

def plot_all_endmembers(true_endmembers, predicted_endmembers_list, augmentation_name):
    """
    Create a subplot for all 4 endmembers comparing true vs. predicted endmembers.
    
    Parameters:
        true_endmembers (numpy array): True endmember matrix (num_endmembers, num_bands).
        predicted_endmembers_list (list of numpy arrays): List of predicted endmembers for multiple runs.
        augmentation_name (str): Name of the augmentation or experiment.
    """
    
    num_endmembers = true_endmembers.shape[0]
    plt.figure(figsize=(16, 10))  # Adjust size for readability

    for endmember_index in range(num_endmembers):
        plt.subplot(3, 2, endmember_index + 1)  # Create a 2x2 grid of subplots
        
        # Plot the true endmember
        plt.plot(true_endmembers[endmember_index] / np.max(true_endmembers[endmember_index]), 
                 label='True Endmember', linewidth=2, color='black')
        
        # Plot all reordered predicted endmembers for this index
        for i, predicted_endmembers in enumerate(predicted_endmembers_list):
            normalized_endmember = predicted_endmembers[endmember_index] / np.max(predicted_endmembers[endmember_index])
            plt.plot(normalized_endmember, linestyle='--', alpha=0.7, label=f'Run {i + 1}' if i < 5 else "")  # Limit legend
        
        plt.title(f'Endmember {endmember_index + 1}')
        plt.xlabel('Spectral Band')
        plt.ylabel('Normalized Reflectance')
        if endmember_index == 0:  # Add legend only for the first plot
            plt.legend(loc='best', fontsize='small')
    
    plt.suptitle(f'Variation of Predicted Endmembers Across Runs ({augmentation_name})', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout for the main title
    plt.show()


path = "./Datasets/Urban5.mat"

try:
    data_org = sio.loadmat(path)
except NotImplementedError:
    data_org = hdf.File(path, 'r')

Y = np.asarray(data_org['Y'], dtype=np.float32)
GT = np.asarray(data_org['GT'], dtype=np.float32)

true_endmembers = GT

num_runs = 25

# Example usage
augmentation_name = "Vertical Flip"
predicted_endmembers_list = []  # Replace with a list of predicted endmember matrices from multiple runs

# Add predicted endmembers from each run to the list
for i in range(num_runs):
    mat_file_path = f'./Urban5/10 images/Results (25 runs, no seed, diff images)/{augmentation_name}/results_run_{i + 1}.mat'
    data = sio.loadmat(mat_file_path)
    predicted_endmembers = data.get('endmembers').T  # Assuming this is your predicted endmember matrix
    
    # Reorder predicted endmembers to match true endmembers
    reordered_predicted_endmembers = reorder_predicted_endmembers(predicted_endmembers, true_endmembers)
    
    # Normalize each reordered endmember
    for m in range(reordered_predicted_endmembers.shape[0]):
        reordered_predicted_endmembers[m, :] = reordered_predicted_endmembers[m, :] / np.max(reordered_predicted_endmembers[m, :])
    
    predicted_endmembers_list.append(reordered_predicted_endmembers)

# Normalize true endmembers
for m in range(true_endmembers.shape[0]):
    true_endmembers[m, :] = true_endmembers[m, :] / np.max(true_endmembers[m, :])

# Plot all 4 endmembers in subplots
plot_all_endmembers(true_endmembers, predicted_endmembers_list, augmentation_name)